In [91]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import poisson
from statsmodels.formula.api import negativebinomial
import numpy as np
import matplotlib.pyplot as plt
import math

#### Read the data files

In [92]:
### All : 90-15, 90-02, 03-15

final_core_counts_90_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core/final_main_gte_10k_core_counts.csv')
final_core_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_90_02/final_main_gte_10k_core_counts_1990_2002.csv')
final_core_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_03_15/final_main_gte_10k_core_counts_2003_2015.csv')

### By Agencies:
## Large
final_core_counts_large_90_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core/final_main_large_cities_core_counts.csv')
final_core_counts_large_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_90_02/final_main_large_cities_core_counts_1990_2002.csv')
final_core_counts_large_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_03_15/final_main_large_cities_core_counts_2003_2015.csv')


## medium
final_core_counts_medium_90_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core/final_main_medium_cities_core_counts.csv')
final_core_counts_medium_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_90_02/final_main_medium_cities_core_counts_1990_2002.csv')
final_core_counts_medium_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_03_15/final_main_medium_cities_core_counts_2003_2015.csv')

## small
final_core_counts_small_90_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core/final_main_small_cities_core_counts.csv')
final_core_counts_small_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_90_02/final_main_small_cities_core_counts_1990_2002.csv')
final_core_counts_small_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_03_15/final_main_small_cities_core_counts_2003_2015.csv')


In [93]:
print(final_core_counts_large_90_02['total_officers'].head())

0    242.0
1    255.0
2    255.0
3    267.0
4    249.0
Name: total_officers, dtype: float64


In [94]:
print(final_core_counts_large_90_02['total_officers'].mode())

0    0.0
dtype: float64


In [99]:
print(final_core_counts_large_90_02['total_officers'].value_counts())
print()
print(final_core_counts_large_90_02['violent_crime'].value_counts())

0.0       69
167.0     26
114.0     25
165.0     24
151.0     23
          ..
1107.0     1
8363.0     1
4104.0     1
1290.0     1
1222.0     1
Name: total_officers, Length: 1280, dtype: int64

295.0     10
628.0      8
532.0      8
1006.0     8
255.0      8
          ..
790.0      1
7146.0     1
7835.0     1
1046.0     1
5178.0     1
Name: violent_crime, Length: 2635, dtype: int64


In [100]:
print(final_core_counts_medium_90_02['total_officers'].value_counts())
print()
print(final_core_counts_medium_90_02['violent_crime'].value_counts())

0.0       197
80.0       89
72.0       84
65.0       83
71.0       81
         ... 
1193.0      1
786.0       1
216.0       1
304.0       1
967.0       1
Name: total_officers, Length: 395, dtype: int64

67.0      37
91.0      34
70.0      34
49.0      32
66.0      32
          ..
771.0      1
1457.0     1
1431.0     1
3225.0     1
1378.0     1
Name: violent_crime, Length: 1239, dtype: int64


In [101]:
print(final_core_counts_small_90_02['total_officers'].value_counts())
print()
print(final_core_counts_small_90_02['violent_crime'].value_counts())

20.0     1069
23.0     1056
21.0     1036
18.0     1031
22.0     1027
         ... 
433.0       1
250.0       1
157.0       1
279.0       1
391.0       1
Name: total_officers, Length: 280, dtype: int64

7.0       696
11.0      695
10.0      690
6.0       681
17.0      676
         ... 
581.0       1
870.0       1
708.0       1
667.0       1
2173.0      1
Name: violent_crime, Length: 810, dtype: int64


In [43]:
def get_req_id_core(df, req_vars):
    df_req = df.loc[:, req_vars]
    return df_req

In [44]:
req_vars = ['ORI', 'YEAR', 'AGENCY', 'placename', 'population', 'violent_crime', 'total_officers', 
            'jail_occupancy_count', 'prison_occupancy_count', 'drug_tot_arrests', 'disorder_arrests_tot_index']

In [45]:
final_core_counts_90_15_req = get_req_id_core(final_core_counts_90_15, req_vars)
final_core_counts_90_02_req = get_req_id_core(final_core_counts_90_02, req_vars)
final_core_counts_03_15_req = get_req_id_core(final_core_counts_03_15, req_vars)

In [61]:
final_core_counts_90_15_req.describe()
final_core_counts_90_15_req['violent_crime'].var()
## variance is much higher than mean

3962039.3716140906

In [47]:
final_core_counts_90_02_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,52314.000000,5.231400e+04,52314.000000,52314.000000,50636.000000,45020.000000,52314.000000,49275.000000
mean,1996.027794,4.941885e+04,346.352716,100.240280,108.489632,213.645691,232.433536,388.236185
std,3.770082,1.647713e+05,2638.665102,634.496628,443.217973,1115.395909,1875.516795,1789.933191
min,1990.000000,1.022000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1993.000000,1.409110e+04,21.000000,21.000000,16.000000,26.000000,17.000000,48.000000
50%,1996.000000,2.235650e+04,55.000000,35.000000,34.000000,59.000000,51.000000,122.000000
75%,1999.000000,4.273525e+04,162.000000,69.000000,81.000000,142.000000,136.000000,292.000000
max,2002.000000,8.024964e+06,174542.000000,41791.000000,17562.000000,47405.000000,146848.000000,85818.000000


In [48]:
final_core_counts_03_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,58067.000000,5.806700e+04,58067.000000,58067.000000,56306.000000,51099.000000,58067.000000,53351.000000
mean,2009.082508,5.206584e+04,231.378907,102.003754,132.833126,248.205327,226.837223,267.022005
std,3.715264,1.377821e+05,1119.200616,448.675134,379.225192,926.928815,824.797683,977.866908
min,2003.000000,4.924800e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2006.000000,1.572990e+04,21.000000,25.000000,24.000000,38.000000,27.000000,33.000000
50%,2009.000000,2.486900e+04,52.000000,41.000000,52.000000,84.000000,76.000000,86.000000
75%,2012.000000,4.743820e+04,140.000000,78.000000,116.000000,191.000000,183.000000,218.000000
max,2015.000000,8.550861e+06,52384.000000,35395.000000,11688.000000,27105.000000,31495.000000,33375.000000


In [49]:
final_core_counts_large_90_15_req = get_req_id_core(final_core_counts_large_90_15, req_vars)
final_core_counts_large_90_02_req = get_req_id_core(final_core_counts_large_90_02, req_vars)
final_core_counts_large_03_15_req = get_req_id_core(final_core_counts_large_03_15, req_vars)

In [50]:
final_core_counts_large_90_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,9120.000000,9.120000e+03,9120.000000,9120.000000,8826.000000,8306.000000,9120.000000,8100.000000
mean,2002.461513,2.906030e+05,2324.130373,670.793311,743.906186,1530.938358,1564.410855,2041.356914
std,7.505209,4.574799e+05,6565.799586,1790.973466,1243.099415,3157.287910,4720.095559,4663.369848
min,1990.000000,2.155800e+04,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000
25%,1996.000000,1.215927e+05,420.000000,183.750000,223.250000,382.000000,273.000000,380.000000
50%,2002.000000,1.709372e+05,872.000000,299.000000,387.000000,710.000000,685.000000,893.000000
75%,2009.000000,2.856638e+05,1957.000000,567.250000,770.000000,1469.750000,1450.000000,1893.250000
max,2015.000000,8.550861e+06,174542.000000,41791.000000,17562.000000,47405.000000,146848.000000,85818.000000


In [51]:
final_core_counts_large_90_02_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,4596.000000,4.596000e+03,4596.000000,4596.000000,4438.000000,4136.000000,4596.000000,4145.000000
mean,1996.006963,2.781723e+05,2785.678851,660.714317,693.908067,1431.934478,1671.479765,2388.373703
std,3.733557,4.979472e+05,8505.494440,2052.826650,1349.003479,3430.919996,6126.271827,5716.664093
min,1990.000000,2.918800e+04,20.000000,0.000000,0.000000,8.000000,0.000000,0.000000
25%,1993.000000,1.124582e+05,444.750000,167.000000,191.000000,322.000000,226.750000,409.000000
50%,1996.000000,1.562776e+05,950.500000,277.000000,341.000000,582.000000,612.000000,968.000000
75%,1999.000000,2.648235e+05,2184.250000,512.500000,658.750000,1280.500000,1430.000000,2120.000000
max,2002.000000,8.024964e+06,174542.000000,41791.000000,17562.000000,47405.000000,146848.000000,85818.000000


In [52]:
final_core_counts_large_03_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,4524.000000,4.524000e+03,4524.000000,4524.000000,4388.000000,4170.000000,4524.000000,3955.000000
mean,2009.018789,3.032315e+05,1855.236295,681.032714,794.474020,1629.135012,1455.637931,1677.669279
std,3.750080,4.119935e+05,3603.305098,1478.361745,1123.796639,2857.174487,2601.163386,3167.866532
min,2003.000000,2.155800e+04,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000
25%,2006.000000,1.338761e+05,402.000000,202.000000,261.000000,457.000000,326.000000,361.000000
50%,2009.000000,1.875338e+05,806.000000,327.000000,443.000000,817.000000,756.000000,826.000000
75%,2012.000000,3.020572e+05,1760.000000,620.250000,854.500000,1643.750000,1474.000000,1636.500000
max,2015.000000,8.550861e+06,52384.000000,35395.000000,11688.000000,27105.000000,31495.000000,33375.000000


In [53]:
final_core_counts_medium_90_15_req = get_req_id_core(final_core_counts_medium_90_15, req_vars)
final_core_counts_medium_90_02_req = get_req_id_core(final_core_counts_medium_90_02, req_vars)
final_core_counts_medium_03_15_req = get_req_id_core(final_core_counts_medium_03_15, req_vars)

In [54]:
final_core_counts_medium_90_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,15187.000000,15187.000000,15187.000000,15187.000000,14669.000000,13197.000000,15187.000000,14077.000000
mean,2002.778297,68914.595720,293.067953,115.690196,149.944372,269.361597,278.487588,423.986148
std,7.508600,16938.463923,308.617140,86.225832,127.355290,221.764333,283.814483,490.630343
min,1990.000000,6141.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1996.000000,56307.500000,97.000000,72.000000,68.000000,119.000000,82.000000,119.000000
50%,2003.000000,65605.000000,197.000000,101.000000,118.000000,208.000000,210.000000,266.000000
75%,2009.000000,80402.700000,368.000000,139.000000,189.000000,342.000000,386.000000,541.000000
max,2015.000000,163406.000000,3880.000000,1621.000000,1702.000000,1886.000000,3967.000000,8534.000000


In [55]:
final_core_counts_medium_90_02_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,7316.000000,7316.000000,7316.000000,7316.000000,7062.000000,6299.000000,7316.000000,6880.000000
mean,1996.026654,63866.599891,321.791279,106.410334,123.852167,229.674710,244.596227,473.287645
std,3.766235,15510.507398,360.126365,80.305533,101.821478,192.781683,278.998861,548.975284
min,1990.000000,6141.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1993.000000,53012.875000,97.000000,64.000000,56.000000,101.000000,63.000000,138.000000
50%,1996.000000,60806.500000,201.000000,91.000000,100.000000,174.000000,168.000000,299.000000
75%,1999.000000,73810.650000,402.000000,130.000000,161.000000,292.000000,338.000000,608.000000
max,2002.000000,155790.000000,3880.000000,1621.000000,1382.000000,1886.000000,3967.000000,8534.000000


In [56]:
final_core_counts_medium_03_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,7871.000000,7871.000000,7871.000000,7871.000000,7607.000000,6898.000000,7871.000000,7197.000000
mean,2009.053869,73606.647237,266.369966,124.315716,174.167214,305.602204,309.989201,376.856190
std,3.720426,16861.479358,248.514695,90.541503,142.951123,239.594805,284.639796,422.163991
min,2003.000000,16381.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2006.000000,59793.700000,97.000000,82.000000,83.000000,142.000000,113.000000,106.000000
50%,2009.000000,70390.400000,192.000000,109.000000,136.000000,240.500000,253.000000,239.000000
75%,2012.000000,85157.100000,346.000000,148.000000,220.000000,389.000000,428.000000,480.000000
max,2015.000000,163406.000000,2152.000000,1584.000000,1702.000000,1795.000000,3052.000000,4791.000000


In [57]:
final_core_counts_small_90_15_req = get_req_id_core(final_core_counts_small_90_15, req_vars)
final_core_counts_small_90_02_req = get_req_id_core(final_core_counts_small_90_02, req_vars)
final_core_counts_small_03_15_req = get_req_id_core(final_core_counts_small_03_15, req_vars)

In [58]:
final_core_counts_small_90_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,86074.000000,86074.000000,86074.000000,86074.000000,83447.000000,74616.000000,86074.000000,80449.000000
mean,2002.961975,22209.954026,68.634930,38.250807,50.421621,80.822263,79.402224,135.151276
std,7.516608,10856.644839,97.256343,31.199873,63.102436,101.273424,102.334263,187.933661
min,1990.000000,1022.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1996.000000,13565.700000,17.000000,21.000000,16.000000,25.000000,17.000000,30.000000
50%,2003.000000,19155.000000,37.000000,31.000000,32.000000,52.000000,48.000000,76.000000
75%,2010.000000,28394.550000,82.000000,47.000000,62.000000,102.000000,103.000000,169.000000
max,2015.000000,91077.000000,2173.000000,1241.000000,1291.000000,4087.000000,1649.000000,5133.000000


In [59]:
final_core_counts_small_90_02_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,40402.000000,40402.000000,40402.000000,40402.000000,39136.000000,34585.000000,40402.000000,38250.000000
mean,1996.030370,20780.395438,73.310504,35.365304,39.331383,65.031864,66.529900,156.191111
std,3.774994,9969.668666,111.379081,29.626448,47.776170,75.328672,94.164957,209.285488
min,1990.000000,1022.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1993.000000,12800.250000,16.000000,19.000000,13.000000,21.000000,14.000000,37.000000
50%,1996.000000,17960.150000,37.000000,29.000000,26.000000,42.000000,38.000000,92.000000
75%,1999.000000,26589.750000,85.000000,44.000000,48.000000,83.000000,83.000000,196.000000
max,2002.000000,76174.600000,2173.000000,897.000000,816.000000,3041.000000,1649.000000,5133.000000


In [60]:
final_core_counts_small_03_15_req.describe()

,YEAR,population,violent_crime,total_officers,jail_occupancy_count,prison_occupancy_count,drug_tot_arrests,disorder_arrests_tot_index
count,45672.000000,45672.000000,45672.000000,45672.000000,44311.000000,40031.000000,45672.000000,42199.000000
mean,2009.093755,23474.558730,64.498861,40.803359,60.216650,94.464465,90.789236,116.080357
std,3.710898,11436.917894,82.559386,32.314614,72.652026,117.531132,107.779004,163.912427
min,2003.000000,4924.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2006.000000,14347.750000,17.000000,22.000000,19.000000,30.000000,22.000000,26.000000
50%,2009.000000,20228.300000,37.000000,33.000000,39.000000,62.000000,59.000000,64.000000
75%,2012.000000,29742.000000,79.000000,50.000000,74.000000,119.000000,119.000000,144.000000
max,2015.000000,91077.000000,1073.000000,1241.000000,1291.000000,4087.000000,1604.000000,4410.000000


#### get the descriptives by agency type too to see if mean and variances differ like this by agency. The variances within each level of prog are higher than the means within each level. These are the conditional means and variances. These differences suggest that over-dispersion is present and that a Negative Binomial model would be appropriate.

#### Joao Santos Silva - even if indeed there is overdispersion, that is not a serious problem unless you want to compute probabilities of particular counts. https://www.statalist.org/forums/forum/general-stata-discussion/general/1430344-panel-negative-binomial-model

#### https://stackoverflow.com/questions/33085184/python-method-for-calculating-conditional-means-and-variances/33085946 - The Poisson regression is calculating the conditional mean, even when there is overdispersion. You could test for overdispersion based on the Poisson estimate. However, with the large number of observations and not too many regressors, I don't see a reason why not to use NegativeBinomial right away instead of pretesting

### Negative Binomial
#### Let's start with final_core_counts_large_90_02 for now to understand how this works. We will compare this with SAS and STATA

In [88]:
final_core_counts_large_90_02['total_officers'].mode()

0    0.490273
dtype: float64

In [90]:
print(final_core_counts_large_90_02['total_officers'].head())

0    0.532236
1    0.537792
2    0.537792
3    0.542607
4    0.535274
Name: total_officers, dtype: float64


In [102]:
# final_core_counts_large_90_02['total_officers'].replace({0: 1}, inplace=True)
# final_core_counts_large_90_02['total_officers'] = np.log(final_core_counts_large_90_02['total_officers'])

#print(final_core_counts_large_90_02['violent_crime'].head())
model = poisson('violent_crime ~ total_officers', data=final_core_counts_large_90_02).fit()
model.summary()

         Current function value: 2335.211104
         Iterations: 35


/Users/salma/anaconda3/envs/us_crime_data_analysis/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:          violent_crime   No. Observations:                 4596
Model:                        Poisson   Df Residuals:                     4594
Method:                           MLE   Df Model:                            1
Date:                Wed, 22 Apr 2020   Pseudo R-squ.:                  0.2929
Time:                        09:06:36   Log-Likelihood:            -1.0733e+07
converged:                      False   LL-Null:                   -1.5178e+07
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          7.6729      0.000   2.49e+04      0.000       7.672       7.673
total_officers     0.0001   1.38e-08   1.08e+04      0.000       0.000       0.000
==================================================================================
"""

In [74]:
large_counts_90_02_pop_ref = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_pop_ref_90_02/final_main_large_cities_core_counts_1990_2002_ref_pop.csv')
print(list(large_counts_90_02_pop_ref))

['ORI', 'AGENCY', 'placename', 'Govt_level', 'place_fips', 'STATEFP', 'CNTY', 'YEAR', 'POP100', 'White_count', 'Black_count', 'Hispanic_count', 'Age1524_WhiteM', 'White_Males_All', 'Age1524_WhiteF', 'White_Females_All', 'Age1524_BlackM', 'Black_Males_All', 'Age1524_BlackF', 'Black_Females_All', 'Hispanic_Males_All', 'Age1524_HispanicM', 'Age1524_HispanicF', 'Hispanic_Females_All', 'Pct_WYM', 'Pct_WYF', 'total_count_county', 'population', 'population_mean', 'murder', 'rape', 'robbery', 'agg_assault', 'simple_assault', 'burglary', 'larceny', 'motor_vehicle_theft', 'violent_crime', 'total_crime', 'property_crime', 'main_crime', 'murder_tot_arrests', 'murder_tot_arrests_black', 'murder_tot_arrests_white', 'rape_tot_arrests', 'rape_tot_arrests_black', 'rape_tot_arrests_white', 'robbery_tot_arrests', 'robbery_tot_arrests_black', 'robbery_tot_arrests_white', 'agg_assault_tot_arrests', 'agg_assault_tot_arrests_white', 'agg_assault_tot_arrests_black', 'larceny_theft_tot_arrests', 'larceny_theft